In [1]:
%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
%env PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin

env: JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
env: PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin


In [2]:
import findspark
import pandas as pd
import numpy as np

import pyspark as psk
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

import time as t 
from datetime import date, datetime, timedelta
import joblib
from joblib import Parallel, delayed

import multiprocessing
from functools import reduce  
from math import modf
import sqlite3
import os
import sys

from delta import * 
from delta.tables import *
from delta import configure_spark_with_delta_pip
# import covalent as ct

import subprocess
import re
import random

from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
# import umap.umap_ as umap
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt

findspark.init()
findspark.find()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# os.environ['PYDEVD_DISABLE_FILE_VALIDATION']=1

spark = SparkSession.builder \
       .master("local[2]") \
       .appName("MyApp") \
       .config("spark.driver.maxResultSize", "20g")\
       .config("spark.driver.memory", "100g")\
       .getOrCreate()
sc = spark.sparkContext

# sqlContext = SQLContext(sc)


#both works
# 1: 
# spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")    #To resolve the error for p1075_38 to_timestamp formating: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '1/2/2019 20:40:00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
# Set Spark SQL legacy time parser policy to LEGACY to handle older date formats
# 2:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
# Increase the max fields in the string representation of a plan
spark.conf.set("spark.sql.execution.arrow.enabled", "false")
spark.conf.set("spark.sql.debug.maxToStringFields", 2000)

ps: /storage/icds/RISE/sw8/anaconda/anaconda3/envs/tensorflow/lib/libuuid.so.1: no version information available (required by /usr/lib64/libblkid.so.1)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/09 00:21:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/09 00:21:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/09 00:21:03 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/04/09 00:21:04 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [3]:
spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data.csv") \
          .createOrReplaceTempView("VINs_data")

spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")

In [4]:
df_all_pop = spark.sql('select * from population')
df_all_pop_p = df_all_pop.toPandas()

In [5]:
# Get all column names
all_columns = df_all_pop.columns

# Print the first 10
print("First 10 column names:")
for col in all_columns[:11]:
    print(col)


First 10 column names:
VIN
CHASSIS_ID
ENGINE_ASSEMB_DATE
ENGINE_ASSEMB_MONTH
VEH_ASSEMB_DATE
VEH_ASSEMB_MONTH
INS_DATE
INS_MONTH
ENGINE_SIZE
ENGINE_HP
VEH_TYPE


In [6]:
# Get all column names
all_columns = df_all_pop.columns

vin_column = [all_columns[0]]

# Include column 2 (index 1)
extra_column = [all_columns[2]]

# Pick columns from index 8 to the end
main_columns = all_columns[8:]

# Combine them
selected_columns = vin_column + extra_column + main_columns

# # Print how many columns are selected
# print(f"Number of selected columns: {len(selected_columns)}")

# # Print their names
# print("Selected column names:")
# for col in selected_columns:
#     print(col)


In [7]:
# # Make sure you only pass PCA-reduced KOLA feature columns
# pca_columns = [col for col in df_final.columns if col.startswith("KOLA_PCA_")]
# df_kmeans_result = apply_kmeans_clustering(df_final[pca_columns], n_clusters=10)

In [8]:
def evaluate_pca_component_range(df, categorical_columns, non_kola_columns, k=10):
    print(" Step 1: Dropping NA rows...")
    df_filtered = df[non_kola_columns + categorical_columns].dropna()
    print(f"    - Filtered shape: {df_filtered.shape}")

    print(" Step 2: One-hot encoding KOLA features...")
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    kola_encoded = encoder.fit_transform(df_filtered[categorical_columns])
    print(f"    - Encoded shape: {kola_encoded.shape}")

    silhouette_scores = []
    n_components = 6
    for n_components in range(6, 16):
        print(f"\n Testing PCA with {n_components} components...")
        pca = PCA(n_components=n_components)
        pca_result = pca.fit_transform(kola_encoded)

        print("    - Clustering with KMeans...")
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(pca_result)

        score = silhouette_score(pca_result, labels)
        silhouette_scores.append(score)
        print(f"    - Silhouette Score: {score:.4f}")

    print("\n Plotting silhouette scores...")
    plt.figure(figsize=(10, 6))
    x_vals = list(range(6, 16))

    plt.plot(x_vals, silhouette_scores, marker='o')
    plt.title("Silhouette Score vs. Number of PCA Components")
    plt.xlabel("Number of PCA Components")
    plt.ylabel("Silhouette Score")
    plt.grid(True)

    # Annotate each point with its score
    for x, y in zip(x_vals, silhouette_scores):
        plt.text(x, y + 0.01, f"{y:.3f}", ha='center', fontsize=9)

    plt.tight_layout()

    # Save the figure as a PDF
    plt.savefig("silhouette_score_vs_pca_components.pdf")

    plt.show()

In [ ]:
vin_column = "VIN"
df = df_all_pop_p.copy()

# Assuming df is your full dataset and selected_columns is already defined
kola_columns = [col for col in selected_columns if "KOLA" in col]
non_kola_columns = [col for col in selected_columns if col not in kola_columns or col == vin_column]

evaluate_pca_component_range(df, categorical_columns=kola_columns, non_kola_columns=non_kola_columns, k=10)


 Step 1: Dropping NA rows...
    - Filtered shape: (321746, 593)
 Step 2: One-hot encoding KOLA features...
    - Encoded shape: (321746, 7845)

 Testing PCA with 6 components...
    - Clustering with KMeans...
    - Silhouette Score: 0.5588

 Testing PCA with 7 components...
    - Clustering with KMeans...
    - Silhouette Score: 0.5431

 Testing PCA with 8 components...
    - Clustering with KMeans...
